In [509]:
import pandas as pd
import numpy as np
import tqdm.notebook as tq
import sklearn.metrics as metrics
import lightgbm as lgb
import re

In [510]:
df = pd.read_csv('dataset.csv')

In [511]:
# df = df.loc[:, ~df.columns.str.contains("svd")]
df = df.rename(columns=lambda x: re.sub("[^A-Za-z0-9_]+", "", x))
df

,Unnamed0,Ticker,Sector,Date,Year,Quarter,Forward_pe,Trailing_pe,Trailing_eps,Market_cap,...,pos_vader,sent_TB,Litigious,Positive,WeakModal,Uncertainty,StrongModal,Negative,Constraining,Target_Class
0,0,CEG,7,2022-05-12,2,1.0,13.589928,0.000000,0.000,1.848790e+10,...,0.296,0.178282,0.850444,0.761099,0.740774,1.837935,0.085738,1.508795,0.544998,0
1,1,DIS,6,2022-05-11,2,2.0,20.804348,96.014626,1.094,1.916197e+11,...,0.259,0.153053,-0.070687,-0.074547,1.266369,-0.142743,1.105179,-0.273462,-0.663678,0
2,2,OXY,10,2022-05-11,2,1.0,23.434483,0.000000,-1.212,6.346657e+10,...,0.245,0.148223,-0.023449,-0.074547,-0.660812,0.048935,-0.169123,-0.605331,-0.628129,0
3,3,WELL,1,2022-05-11,2,1.0,88.054450,85.874400,1.035,4.032598e+10,...,0.285,0.124651,0.708731,0.384470,0.390378,0.496185,2.634342,0.586938,0.118407,0
4,4,XRAY,11,2022-05-11,2,1.0,12.023511,20.152473,1.902,8.243193e+09,...,0.290,0.115617,-0.023449,0.360930,-0.485614,0.304507,-0.678844,1.238384,-0.165988,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,1945,LULU,3,2021-03-31,0,4.0,32.805410,45.670610,6.661,3.932629e+10,...,0.275,0.106794,0.803206,2.055762,0.565576,1.774042,-0.423983,1.029429,1.433730,0
1946,1946,MKC,4,2021-03-30,1,1.0,32.558067,35.792553,2.820,2.698570e+10,...,0.308,0.125826,0.661494,3.338655,-0.135217,0.464239,0.085738,3.020641,1.469280,0
1947,1947,FDS,8,2021-03-30,1,2.0,27.438377,35.348053,10.530,1.407757e+10,...,0.315,0.137788,-0.472205,-0.604182,-0.485614,-0.909457,-0.423983,-0.457834,-0.983622,0
1948,1948,DRI,3,2021-03-25,1,3.0,14.899408,18.075770,6.995,1.640223e+10,...,0.253,0.135263,-0.826486,-0.486485,-0.135217,-0.621940,-0.169123,-0.445542,-0.414833,0


In [512]:
# print(len(df[df.Target_Class==0]))
# print(len(df[df.Target_Class==1]))
df = df.loc[:, ~df.columns.str.contains("Unnamed")]

In [513]:
from sklearn.model_selection import train_test_split
X = df.drop(['Ticker','Date', 'Target', 'Target_Class'], axis = 1)

y = df['Target_Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [514]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [515]:
pred_y = model.predict(X_test)
accuracy = metrics.accuracy_score(y_test, pred_y)
print("Accuracy: ", accuracy)

Accuracy:  0.782051282051282


In [516]:
clf = lgb.LGBMClassifier(boosting_type='gbdt', learning_rate=0.1, n_estimators=100, random_state=42, feature_fraction=0.8)
clf.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8


LGBMClassifier(feature_fraction=0.8, random_state=42)

In [518]:
clf.booster_.feature_importance()
fea_imp_ = pd.DataFrame({'cols':X_train.columns, 'fea_imp':clf.feature_importances_})
fea_imp_.loc[fea_imp_.fea_imp > 0].sort_values(by=['fea_imp'], ascending = False)

,cols,fea_imp
9,svd_3,498
8,svd_2,485
18,sent_TB,479
10,svd_4,473
16,neu_vader,445
7,svd_1,440
24,Negative,436
13,d_c,432
25,Constraining,432
6,Market_cap,403


In [506]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
dtree = DecisionTreeClassifier(random_state=42)
dtree.fit(X_train, y_train)
pred_y = dtree.predict(X_test)
accuracy = metrics.accuracy_score(y_test, pred_y)
print(f'Decision Tree accuracy score: {accuracy}')

knn = KNN()
knn.fit(X_train, y_train)
pred_y_knn = knn.predict(X_test)
accuracy = metrics.accuracy_score(y_test, pred_y_knn)
print(f'KNN accuracy score: {accuracy}')

y_pred=clf.predict(X_test)
accuracy=metrics.accuracy_score(y_pred, y_test)
print(f'LightGBM Model accuracy score: {accuracy}')

Decision Tree accuracy score: 0.6794871794871795
KNN accuracy score: 0.764102564102564
LightGBM Model accuracy score: 0.7743589743589744
